## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
import os
os.chdir('/drive/My Drive/W.A.S.D Warriors')

In [ ]:
!pip install folium==0.2.1
!pip install imgaug==0.2.5

!pip install --force-reinstall --user --no-warn-script-location -q https://ai-camp.s3-us-west-2.amazonaws.com/AiCampEval-1.7-py3-none-any.whl

     |████████████████████████████████| 71kB 5.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 563kB 6.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/31/48/c8/ca3345e8582a078de94243996e148377ef66fdb845557bae0b
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
     |████████████████████████████████| 17.3MB 6.9MB/s 
     |████████████████████████████████| 2.0MB 37.6MB/s 


##Use AiCampEval to import eval_submit
Reset if unable to get below

In [ ]:
from AiCampEval import eval_submit

In [ ]:
import os

base_dir = 'data/trainset_11classes_0_00000'

# Directory to our training data
train_folder = os.path.join(base_dir, 'train')

# Directory to our validation data
val_folder = os.path.join(base_dir, 'val')

##Load Model



In [ ]:
from keras.models import load_model
import os
import time

os.chdir('/drive/My Drive/W.A.S.D Warriors')

st = time.time()
model_path = 'Xcep15_0.79(previous).hdf5'
model = load_model( model_path )
print('Time taken: ' + str(time.time()-st))

Time taken: 83.43276333808899


##PreProcessing

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'data/trainset_11classes_0_00000'
train_folder = os.path.join(base_dir, 'train_new_combined')

num_models = 1

# Batch size
bs = 32

# All images will be resized to this value
image_size = (299, 299)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, # This is the source directory for training images 
    target_size=image_size, # All images will be resized to value set in image_size
    batch_size=bs,
    class_mode='categorical')

print(train_generator.class_indices)

val_datagen = ImageDataGenerator(rescale=1./255)

def generate_generator_multiple(generator, num_model, directory, bs, image_size):
  generator1 = generator.flow_from_directory(
      directory= directory, # This is the source directory for training images 
      target_size=image_size, # All images will be resized to value set in image_size
      batch_size=bs,
      class_mode='categorical')
  
  while True:
    generator = generator1.next()
    x_input = []
    
    for i in range(num_model):
      x_input.append(generator[0])
    
    y_input = generator[1]
    
    yield x_input, y_input  #Yield both images and their mutual label


# Flow validation images in batches of 32 using val_datagen generator
print("Preparing generator for validation dataset")
val_generator = generate_generator_multiple(val_datagen, num_models, val_folder, bs, image_size)

Found 1441 images belonging to 15 classes.
{'ChairPose': 0, 'ChestBump': 1, 'ChildPose': 2, 'Dabbing': 3, 'EaglePose': 4, 'HandGun': 5, 'HandShake': 6, 'HighKneel': 7, 'HulkSmash': 8, 'KoreanHeart': 9, 'KungfuCrane': 10, 'KungfuSalute': 11, 'Salute': 12, 'Spiderman': 13, 'WarriorPose': 14}
Preparing generator for validation dataset


In [ ]:
import numpy as np
from PIL import Image

ensemble = False
num_models = 1

'''
This is a helper function to preprocess the input before sending it through my network. 
It performs the following steps:
1. Resizes each image to (224,224,3), which are the image dimensions my model was trained on.
2. Normalizes the range of each pixel's value from [0-255] to [0-1].
3. Reshapes each array such that it has a batch dimension: from  (224,224,3) -> (1,224,224,3), 
because my model.predict function expects a batch dimension.

Implement your own preprocessing function according to your model's needs.
''' 
def preprocess_arrays(list_of_np_arrays):
    target_size = image_size
    pil_images = [ Image.fromarray(arr.astype('uint8'), 'RGB') for arr in list_of_np_arrays ]
    resized_pil_images = [img.resize( target_size, Image.NEAREST ) for img in pil_images]
    resized_images = [np.array( img ) for img in resized_pil_images]
    
    if ensemble == False:
      preprocessed_images = [np.expand_dims(x, axis=0) / 255. for x in resized_images]
      return preprocessed_images
    else:
      preprocessed_images = [np.expand_dims(x, axis=0) / 255. for x in resized_images]
      x_input = []

      for i in range(len(preprocessed_images)):
        x = [preprocessed_images[i] for _ in range(num_models)]
        x_input.append(x)

      return x_input

'''
This function accepts a list of numpy array images as input and outputs a list of prediction strings 
corresponding to each image in the input list.
IMPORTANT: you have to implement this function.
''' 
def evaluate_images(list_of_np_arrays):
    # Swap the key and value in the class_indices
    label_dict = dict((v,k) for k,v in (train_generator.class_indices).items())
    # Run through the list_of_np_arrays to perform any preprocessing you need
    preprocessed_imgs = preprocess_arrays( list_of_np_arrays )
    # Get a list of softmax_vectors by passing in the preprocessed_imgs to model.predict()
    softmax_vectors = [ model.predict( x )[0]  for x in preprocessed_imgs]
    # Get the predicted_class_indices by running each of the softmax_vectors through np.argmax()
    predicted_class_indices = [ np.argmax( x ) for x in softmax_vectors ]
    # Convert the list of class_indices to a list of predictions (in str)
    # predictions is a list of labels: ['KoreanHeart', 'KoreanHeart','ChairPose',.......]
    predictions = [ label_dict[k] for k in predicted_class_indices ]
    return predictions

##Submission

In [ ]:
TEAM_ID = "W.A.S.D Warriors"
SUBMISSION_TYPE = "thefinaltest_4_88888"

eval_submit(evaluate_images, SUBMISSION_TYPE, TEAM_ID )


Beginning file download of test images

Time taken for download: 67.880s

Predicting batch 1/4...

Predicting batch 2/4...

Predicting batch 3/4...

Predicting batch 4/4...

Total time taken for model evaluation: 153.503s

Submitting predictions
{'accuracy': 0.7812,
 'submission_time': '2019-06-13 17:41:21.554828+08:00',
 'submission_type': 'thefinaltest_4_88888',
 'team_id': 'W.A.S.D Warriors'}


For normal val_generator evaluation

In [ ]:
scores = model.evaluate_generator(val_generator, steps=253 // 32 + 1, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])